## Welcome to your notebook.


#### Gets all users in arcgis and adds their data to users.csv

In [1]:
# Logger script that logs errors from ArcGIS Enterprise/Online - Migrating User Content and Groups.py script
from arcgis.gis import GIS
import logging
import pathlib
import os
import datetime


class Logger(object):
    def __init__(self):
        self.outputToFile = True

    def setLogging(self, logFileName):

        # Change the working directory to our script's location
        currentScriptDirectory = os.path.dirname(os.path.realpath(__file__))
        os.chdir(currentScriptDirectory)

        # Create the logs folder if it doesn't already exist
        logsDirectory = pathlib.Path.cwd()
        logsDirectory = logsDirectory / 'logs'
        logsDirectory.mkdir(parents=True, exist_ok=True)

        # Get the log file with the date appended to it and set up basic logging
        logFilePath = logsDirectory / f'{logFileName} Log File.txt'
        if os.path.exists(logFilePath):
           os.remove(logFilePath)
        logging.basicConfig(filename=str(logFilePath), level=logging.INFO,
                            format='%(asctime)s - %(levelname)s - %(message)s')
        self.logger = logging.getLogger()

    def setLevel(self, level):
        if level.upper() == "DEBUG":
            self.logger.setLevel(logging.DEBUG)
        elif level.upper() == "INFO":
            self.logger.setLevel(logging.INFO)
        elif level.upper() == "WARN":
            self.logger.setLevel(logging.WARN)
        elif level.upper() == "ERROR":
            self.logger.setLevel(logging.ERROR)
        elif level.upper() == "CRITICAL":
            self.logger.setLevel(logging.CRITICAL)

    def debug(self, message, exception=None):
        output = self.formatMessage(message, exception)
        if logging.getLevelName("DEBUG") >= self.logger.level:
            arcpy.AddMessage(output)
            if self.outputToFile:
                self.logger.debug(output)

    def info(self, message, exception=None):
        output = self.formatMessage(message, exception)
        if logging.getLevelName("INFO") >= self.logger.level:
            arcpy.AddMessage(output)
            if self.outputToFile:
                self.logger.info(output)

    def warn(self, message, exception=None):
        output = self.formatMessage(message, exception)
        if logging.getLevelName("WARN") >= self.logger.level:
            arcpy.AddWarning(output)
            if self.outputToFile:
                self.logger.warn(output)

    def error(self, message, exception=None):
        output = self.formatMessage(message, exception)
        if logging.getLevelName("ERROR") >= self.logger.level:
            arcpy.AddWarning(output)
            if self.outputToFile:
                self.logger.error(output)

    def critical(self, message, exception=None):
        output = self.formatMessage(message, exception)
        if logging.getLevelName("CRITICAL") >= self.logger.level:
            arcpy.AddError(output)
            if self.outputToFile:
                self.logger.critical(output)

    def formatMessage(self, message, exception):
        if exception:
            return f"{message}\n{exception}"

        return message
    
    
# This script writes all built-in user accounts and their associated groups, and add-on licenses to a CSV file
# The CSV will be used in the 'ArcGIS  Online - Migrating User Content and Groups.py

import csv, json, datetime
import requests 
from arcgis.gis import GIS

# Variables
portal = 'https://calpoly.maps.arcgis.com/'                                                             # AGOL URL
username = 'dle79_calpoly'                                                          # Admin Username
password = 'Food3663!'                                                                         # Admin Password
csvFile = r'/arcgis/home/users.csv'       # Output CSV File

# # Set Logging
# logger = Logger()
# logger.setLogging("Built-In Users")
# logger.setLevel('ERROR')

def convert_time(time_value):
    # remove sub-second values
    converted_time = int(time_value//1000)
    timestamp = datetime.datetime.fromtimestamp(converted_time)
    converted_time = timestamp.strftime('%Y-%m-%d %H:%M:%S')
    return converted_time


if __name__ == '__main__':
    gis = GIS(portal, username, password)
    print(f'Logged in as: {gis.properties.user.username}')

    # Get Token
    token = gis._con.token

    users = gis.users.search(max_users=6500, exclude_system=True)
    user_list = []
    for user in users:
        if user.provider != 'enterprise':
            print(f'Gathering info for user: {user.username}')
            if user.created > 0:
                created = convert_time(user.created)
            else:
                created = ''
            if user.lastLogin > 0:
                last_login = convert_time(user.lastLogin)
            else:
                last_login = 'Never'
            try:
                applist = []
                url = f'https://www.arcgis.com/sharing/rest/community/users/{user.username}/provisionedListings'
                params = {'f': 'pjson', 'token': token}
                r = requests.post(url, data=params)
                response = json.loads(r.content)
                for item in response['provisionedListings']:
                    applist.append(item['title'])
                applist = "; ".join(applist)
            except Exception as e:
                logger.error(f"\tError occurred getting provisions for {user.username}. {e}")
                applist = ''

            grouplist = []
            for g in user.groups:
                try:
                    grouplist.append(g.title)
                except:
                    pass
            grouplist = "; ".join(grouplist)


            # Configure first and last name
            try:
                firstName = user.firstName
            except:
                firstName = user.fullName.split(" ")[0]
            try:
                lastName = user.lastName
            except:
                lastName = user.fullName.split(" ")[-1]

            # Configure IDP Username
            idpUsername = user.email

            if user.level == '1':
                user.role = 'viewer'
                user_list.append(
                    [user.username, idpUsername, firstName, lastName, user.email, created, last_login,
                     user.level, user.userLicenseTypeId, user.role, user.storageUsage, applist, grouplist])
            else:
                user_list.append([user.username, idpUsername, firstName, lastName, user.email, created, last_login,
                                  user.level, user.userLicenseTypeId, user.role,  user.storageUsage, applist, grouplist])

            with open(csvFile, 'w', newline='') as myFile:
                writer = csv.writer(myFile)
                writer.writerow(['Username', 'IDP Username', 'First Name', 'Last Name', 'Email', 'Created',
                                 'Last Login', 'Level', 'License Type', 'Role', 'Storage Usage','Apps', 'Groups'])
                for user in user_list:
                    writer.writerow(user)

Logged in as: dle79_calpoly
Gathering info for user: aaahern_CalPoly
Gathering info for user: aaaresta_CalPoly
Gathering info for user: aabeaver_CalPoly
Gathering info for user: aabsy_CalPoly
Gathering info for user: aabuaita_CalPoly_2023
Gathering info for user: aabude_CalPoly
Gathering info for user: aadalal_CalPoly
Gathering info for user: aadams55_CalPoly
Gathering info for user: aalander_CalPoly
Gathering info for user: aala_CalPoly
Gathering info for user: aalbanes_CalPoly
Gathering info for user: aalcal07_CalPoly
Gathering info for user: aalghura_CalPoly
Gathering info for user: aalvar59
Gathering info for user: aarell19_CalPoly
Gathering info for user: aarendt_CalPoly
Gathering info for user: aasaro_CalPoly
Gathering info for user: aau05_CalPoly
Gathering info for user: aau05_CalPoly6
Gathering info for user: aaugusto_CalPoly
Gathering info for user: aavalade_CalPoly
Gathering info for user: aayuso_CalPoly
Gathering info for user: abaehr_CalPoly
Gathering info for user: abbarre

Gathering info for user: AGOL_CalPoly4
Gathering info for user: AGOL_CalPoly40
Gathering info for user: AGOL_CalPoly41
Gathering info for user: AGOL_CalPoly42
Gathering info for user: AGOL_CalPoly43
Gathering info for user: AGOL_CalPoly44
Gathering info for user: AGOL_CalPoly45
Gathering info for user: AGOL_CalPoly46
Gathering info for user: AGOL_CalPoly47
Gathering info for user: AGOL_CalPoly48
Gathering info for user: AGOL_CalPoly49
Gathering info for user: AGOL_CalPoly5
Gathering info for user: AGOL_CalPoly50
Gathering info for user: AGOL_CalPoly6
Gathering info for user: AGOL_CalPoly7
Gathering info for user: AGOL_CalPoly8
Gathering info for user: AGOL_CalPoly9
Gathering info for user: agome104_CalPoly
Gathering info for user: agonz294_CalPoly
Gathering info for user: agoodin_CalPoly
Gathering info for user: agoshtas_CalPoly
Gathering info for user: agpaulse_CalPoly
Gathering info for user: agreen29_CalPoly
Gathering info for user: agreen30_CalPoly
Gathering info for user: agreen35

Gathering info for user: amavor_CalPoly
Gathering info for user: amburnet_CalPoly
Gathering info for user: amcairns_calPoly
Gathering info for user: amcbratn_CalPoly
Gathering info for user: amcconke_CalPoly
Gathering info for user: amccul04_CalPoly
Gathering info for user: amcfar02_CalPoly
Gathering info for user: amdai_CalPoly
Gathering info for user: amdalton_CalPoly
Gathering info for user: amdisbro_CalPoly
Gathering info for user: amdowlin_CalPoly
Gathering info for user: amen_CalPoly
Gathering info for user: ameyer39_CalPoly_2023
Gathering info for user: amforman_CalPoly
Gathering info for user: amiche05_CalPoly
Gathering info for user: amitch14_CalPoly
Gathering info for user: amjewell_CalPoly_2023
Gathering info for user: amkuhn_CalPoly
Gathering info for user: ammaccar_CalPoly
Gathering info for user: ammcgraw_CalPoly
Gathering info for user: ammontgo_CalPoly
Gathering info for user: amogetta_CalPoly
Gathering info for user: amogetta_NR418_Calpoly
Gathering info for user: amon

Gathering info for user: auyesugi_CalPoly
Gathering info for user: avalle01_CalPoly
Gathering info for user: avansind_CalPoly
Gathering info for user: avansind_CalPoly7
Gathering info for user: avbarrer_CalPoly
Gathering info for user: avclark_CalPoly
Gathering info for user: aventer_CalPoly
Gathering info for user: avesely_CalPoly
Gathering info for user: avinayak_CalPoly
Gathering info for user: avityuko_CalPoly
Gathering info for user: avollucc_CalPoly
Gathering info for user: avquinn_CalPoly
Gathering info for user: awagne14_CalPoly
Gathering info for user: awayte_CalPoly_2023
Gathering info for user: awburr_CalPoly
Gathering info for user: awchiu_CalPoly
Gathering info for user: awclarke_Calpoly
Gathering info for user: awebenba_CalPoly
Gathering info for user: aweiss05_CalPoly
Gathering info for user: aweizer_CalPoly
Gathering info for user: awill100_CalPoly
Gathering info for user: awill109_CalPoly
Gathering info for user: awill148_CalPoly
Gathering info for user: awilso76_CalPo

Gathering info for user: bmquan_CalPoly
Gathering info for user: bmuirhea_CalPoly
Gathering info for user: bnakai_CalPoly
Gathering info for user: bnakaji_CalPoly
Gathering info for user: bnauman_CalPoly
Gathering info for user: bndesai_CalPoly
Gathering info for user: bnerhus_CalPoly
Gathering info for user: bnewbold_CalPoly
Gathering info for user: bnguy128_CalPoly
Gathering info for user: bnguy141_CalPoly
Gathering info for user: bnguy185_ITP
Gathering info for user: bnguy227_CalPoly
Gathering info for user: bnlew_CalPoly
Gathering info for user: bnormand_CalPoly
Gathering info for user: bnowicki_calpoly
Gathering info for user: bnturk_CalPoly
Gathering info for user: boneil03_CalPoly
Gathering info for user: bosbor01_CalPoly
Gathering info for user: botashev_CalPoly
Gathering info for user: bourke_CalPoly
Gathering info for user: bowoodwo_CalPoly
Gathering info for user: bparke07_3Dnr418
Gathering info for user: bparke07_CalPoly
Gathering info for user: bpera_CalPoly
Gathering info

...

Gathering info for user: rsgardne_CalPoly
Gathering info for user: rshah28_CalPoly
Gathering info for user: rshahzad_CalPoly
Gathering info for user: rshenkir_CalPoly
Gathering info for user: rshirais_CalPoly
Gathering info for user: rsibul_CalPoly
Gathering info for user: rsingh34_CalPoly
Gathering info for user: rsmody_CalPoly
Gathering info for user: rsomani_CalPoly
Gathering info for user: rsplitge_CalPoly
Gathering info for user: rspradha_CalPoly
Gathering info for user: rstucke_CalPoly
Gathering info for user: rsurane_CalPoly
Gathering info for user: rswang_CalPoly
Gathering info for user: rtenerow_CalPoly
Gathering info for user: rtgoodwi_CalPoly
Gathering info for user: rtito_CalPoly
Gathering info for user: rtlarson_CalPoly
Gathering info for user: rtmcgove_CalPoly
Gathering info for user: rtolles_CalPoly
Gathering info for user: rtoombs_CalPoly
Gathering info for user: rtrainor_CalPoly
Gathering info for user: rtrapp_SLOAG
Gathering info for user: rturpin_CalPoly
Gathering in

Gathering info for user: sgupta13_CalPoly
Gathering info for user: shalusch_CalPoly
Gathering info for user: shambale_CalPoly
Gathering info for user: shan17_CalPoly
Gathering info for user: sharve05_CalPoly
Gathering info for user: sheedy_CalPoly
Gathering info for user: sheetz_CalPoly
Gathering info for user: sheimerd_CalPoly
Gathering info for user: shenry01_CalPoly
Gathering info for user: shereefu_CalPoly
Gathering info for user: shgross_CalPoly
Gathering info for user: sho25_CalPoly
Gathering info for user: shpatric_calpoly
Gathering info for user: shsankar_CalPoly
Gathering info for user: shuang53_CalPoly
Gathering info for user: shuberth_CalPoly
Gathering info for user: shumble_CalPoly
Gathering info for user: shurley_CalPoly
Gathering info for user: sidchadwickcc_CalPoly
Gathering info for user: sierrablackford7_CRMW
Gathering info for user: siglehea_CalPoly
Gathering info for user: sihudson_CalPoly
Gathering info for user: silaw_CalPoly
Gathering info for user: siscott_CalPol

Gathering info for user: sriver11_CalPoly
Gathering info for user: srlevin_CalPoly
Gathering info for user: srmoser_CalPoly
Gathering info for user: srmuirhe_CalPoly
Gathering info for user: srober40_CalPoly
Gathering info for user: srodri40_CalPoly
Gathering info for user: srogel_CalPoly
Gathering info for user: srrice0
Gathering info for user: srtucker_CalPoly
Gathering info for user: srudman_CalPoly
Gathering info for user: ssaik_CalPoly
Gathering info for user: ssalaz_CalPoly
Gathering info for user: ssandrew_CalPoly
Gathering info for user: ssanthos_CalPoly
Gathering info for user: sscanlan_CalPoly
Gathering info for user: sschombu_CalPoly
Gathering info for user: sschwenk_CalPoly
Gathering info for user: ssebor_CalPoly
Gathering info for user: ssellami_CalPoly
Gathering info for user: sselmi_ITP
Gathering info for user: sserra01_CalPoly
Gathering info for user: sshimer_CalPoly
Gathering info for user: sshimer_CalPoly9
Gathering info for user: ssing123_CalPoly
Gathering info for u

Gathering info for user: tkim54_CalPoly
Gathering info for user: tklir_CalPoly
Gathering info for user: tkshatri_CalPoly
Gathering info for user: tksinnot_CalPoly
Gathering info for user: tkwan06_CalPoly
Gathering info for user: tkwardel_CalPoly
Gathering info for user: tkyizom_CalPoly
Gathering info for user: tlarso01_CalPoly
Gathering info for user: tlavine_CalPoly
Gathering info for user: tlballew_CalPoly_2023
Gathering info for user: tldale_CalPoly
Gathering info for user: tldang_CalPoly
Gathering info for user: tle156_ITP
Gathering info for user: tlee164_CalPoly
Gathering info for user: tli30_CalPoly
Gathering info for user: Tljones_CalPoly
Gathering info for user: tlshore_CalPoly
Gathering info for user: tmanahan_CalPoly
Gathering info for user: tmastin
Gathering info for user: tmayeda_CalPoly
Gathering info for user: tmcnutt_CalPoly
Gathering info for user: tmelillo_CalPoly
Gathering info for user: tmille42_CalPoly
Gathering info for user: tmiric_CalPoly
Gathering info for user:

Gathering info for user: wbrook01_CalPoly
Gathering info for user: wburdick_CalPoly
Gathering info for user: wcharter_CalPoly
Gathering info for user: wchau01_CalPoly
Gathering info for user: wclark07_CalPoly
Gathering info for user: wcoplan_CalPoly
Gathering info for user: wdalton_CalPoly
Gathering info for user: wdietz_CalPoly
Gathering info for user: wdu_CalPoly
Gathering info for user: weis_CalPoly
Gathering info for user: wesulliv_CalPoly
Gathering info for user: wfallini_CalPoly
Gathering info for user: wfdaniel_CalPoly
Gathering info for user: wfnorton_CalPoly
Gathering info for user: wfry_CalPoly
Gathering info for user: wgeoris_CalPoly
Gathering info for user: wguermas_CalPoly
Gathering info for user: wguillen_CalPoly
Gathering info for user: wgwallac_CalPoly
Gathering info for user: wheelerq_CalPoly
Gathering info for user: whitson
Gathering info for user: whogg_CalPoly
Gathering info for user: whuang30_CalPoly
Gathering info for user: wieasley_CalPoly
Gathering info for user

#### Cleaning Data

In [2]:
pip install pandasql

  Preparing metadata (setup.py) ... done
  Obtaining dependency information for tzdata>=2022.1 from https://files.pythonhosted.org/packages/a3/fb/52b62131e21b24ee297e4e95ed41eba29647dad0e0051a92bb66b43c70ff/tzdata-2023.4-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 16.3 MB/s eta 0:00:00
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=458abe71d119798b08751b56dc9a01e49088c924354e5e9e245f8e60b35df192
  Stored in directory: /home/arcgis/.cache/pip/wheels/63/e8/ec/75b1df467ecf57b6ececb32cb16f4e86697cbfe55cb0c51f07
Successfully built pandasql
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch-geometric 1.7.2 requires python-louvain, which is not installed.
Note: you may need to restart the kernel to use updated packages.


In [8]:

import pandas as pd
import pandasql as ps

#Import users spreadsheet as a data table in python
df_users = pd.read_csv("/arcgis/home/users.csv")

#Rename columns to be only one word
df_usersclean = df_users.rename(columns = {"Last Login": "LastLogin", 
                                               "First Name": "FirstName",
                                               "Last Name": "LastName",
                                               "Storage Usage": "StorageUsage"
                                               })

#### Seperate No Login Users

In [10]:

#gets all the info of those who haven't logged in ever and their account was created before 7/1/2023
df_noLogin = ps.sqldf("SELECT * FROM df_usersclean WHERE Created < '2023-07-01' AND LastLogin = 'Never'", globals())



#### Users with Multiple accounts

In [11]:
#creates list of emails associated multiple accounts
df_multAccount = ps.sqldf("""SELECT Email, COUNT(*) AS NumAccounts
FROM df_usersclean
GROUP BY Email
HAVING COUNT(*) > 1
ORDER BY NumAccounts""", globals())

In [12]:
df_multAccountCount = ps.sqldf("""SELECT SUM(NumAccounts)
FROM df_multAccount""", globals())

In [13]:
#gets info of all accounts with multiple users
df_multAccountInfo = ps.sqldf("""SELECT * , df_usersclean.Email AS realEmail
FROM df_usersclean 
LEFT JOIN df_multAccount 
ON df_usersclean.Email = df_multAccount.Email""")


In [14]:
print(df_multAccountCount)

   SUM(NumAccounts)
0              1200


In [15]:
df_usersMult = ps.sqldf("""SELECT *
FROM df_multAccountInfo
WHERE Username LIKE '%_calpoly'""", globals())
print(df_usersMult)

              Username          IDP Username FirstName  LastName  \
0      aaahern_CalPoly   aaahern@calpoly.edu     Allie     Ahern   
1     aaaresta_CalPoly  aaaresta@calpoly.edu    Amelia  Aarestad   
2     aabeaver_CalPoly  aabeaver@calpoly.edu    Audrey    Beaver   
3        aabsy_CalPoly     aabsy@calpoly.edu        AJ      Absy   
4       aabude_CalPoly    aabude@calpoly.edu     Arden     Abude   
...                ...                   ...       ...       ...   
5049    ztoler_CalPoly    ztoler@calpoly.edu   Zachery     Toler   
5050   ztopper_CalPoly   ztopper@calpoly.edu   Zachary    Topper   
5051   zwarsen_CalPoly   zwarsen@calpoly.edu      Zane    Warsen   
5052  zwillman_CalPoly  zwillman@calpoly.edu      Zane   Willman   
5053  zzwahlen_CalPoly  zzwahlen@calpoly.edu      Zach   Zwahlen   

                   Email              Created            LastLogin  Level  \
0                   None  2022-01-04 17:44:00  2022-09-28 22:01:58      2   
1                   None  202

#### Not @calpoly.edu

In [16]:
df_notCalPolyEDU = ps.sqldf("""SELECT *
FROM df_usersclean
WHERE Email NOT LIKE "%@calpoly.edu"
ORDER BY Created
""", globals())

print(df_notCalPolyEDU)

               Username                     IDP Username  FirstName  \
0     ncleyshon_CalPoly              ncleyshon@gmail.com     Nicola   
1     lanceknox_CalPoly            lance.knox@utexas.edu      Lance   
2      knehrenz_CalPoly           katienehrenz@gmail.com      Katie   
3      mridland_CalPoly                mridland@esri.com       Mike   
4      dkovacic_CalPoly            dkovacic@illinois.edu      David   
..                  ...                              ...        ...   
78     mmalecha_cp_tamu           mmalecha@arch.tamu.edu    Matthew   
79          syu_cp_tamu                syu@arch.tamu.edu       Siyu   
80   jmasterson_cp_tamu         jmasterson@arch.tamu.edu     Jamie    
81  development_calpoly  development@returntofreedom.org  Chantalle   
82       nstick_CalPoly                nstick@icloud.com      Nevia   

     LastName                            Email              Created  \
0     Leyshon              ncleyshon@gmail.com  2014-10-03 21:57:44   
1    

#### Data Summed Up

In [17]:
df_marked = ps.sqldf("""SELECT *,

COALESCE(NumAccounts, 1) AS numberAccounts,

CASE 
WHEN Created < '2023-07-01' AND LastLogin = 'Never' THEN 'OldNoLoginUser'
ELSE 'LoggedInUser'
END AS UserUse,

CASE
WHEN numAccounts > 1 AND Username LIKE '%_calpoly' THEN 'MultAccount_CalPoly'
WHEN numAccounts > 1  THEN 'MultAccountNOT_CalPoly'
ELSE 'SingleAccount'
END AS multAccount,

CASE
WHEN realEmail NOT LIKE "%@calpoly.edu" THEN 'not@CalPoly.edu'
ELSE "@calpoly.edu"
END AS EmailType

FROM df_multAccountInfo""", globals())

df_marked = df_marked.drop(columns = {"NumAccounts", "Email"})

print(df_marked)

                   Username          IDP Username FirstName  LastName  \
0           aaahern_CalPoly   aaahern@calpoly.edu     Allie     Ahern   
1          aaaresta_CalPoly  aaaresta@calpoly.edu    Amelia  Aarestad   
2          aabeaver_CalPoly  aabeaver@calpoly.edu    Audrey    Beaver   
3             aabsy_CalPoly     aabsy@calpoly.edu        AJ      Absy   
4     aabuaita_CalPoly_2023  aabuaita@calpoly.edu     Aydan   Abuaita   
...                     ...                   ...       ...       ...   
6253         ztoler_CalPoly    ztoler@calpoly.edu   Zachery     Toler   
6254        ztopper_CalPoly   ztopper@calpoly.edu   Zachary    Topper   
6255        zwarsen_CalPoly   zwarsen@calpoly.edu      Zane    Warsen   
6256       zwillman_CalPoly  zwillman@calpoly.edu      Zane   Willman   
6257       zzwahlen_CalPoly  zzwahlen@calpoly.edu      Zach   Zwahlen   

                  Created            LastLogin  Level          License Type  \
0     2022-01-04 17:44:00  2022-09-28 22:01:

In [18]:
print(ps.sqldf("""SELECT *
FROM df_marked
WHERE multAccount = 'MultAccountNOT_CalPoly'
GROUP BY realEmail""", globals()))

                   Username          IDP Username  FirstName   LastName  \
0    NR218_2228_05_aalcal07  aalcal07@calpoly.edu   Angelica     Alcala   
1            aau05_CalPoly6     aau05@calpoly.edu      Amber         Au   
2          abergey_CalPoly6   abergey@calpoly.edu  Alexander     Bergey   
3       NR218_2224_acazar03  acazar03@calpoly.edu       Alan    Cazares   
4       NR218_2218_achhetri  achhetri@calpoly.edu      Ashim    Chhetri   
..                      ...                   ...        ...        ...   
515       wraduzin_CalPoly7  wraduzin@calpoly.edu    William  Raduziner   
516  NR218_05_2238_zaparson  zaparson@calpoly.edu       Zack    Parsons   
517        zbarber_CalPoly8   zbarber@calpoly.edu    Zachary     Barber   
518           Y_2228_zbrant    zbrant@calpoly.edu     Zander      Brant   
519       NR418_2214_ztoler    ztoler@calpoly.edu    Zachery      Toler   

                 Created            LastLogin  Level          License Type  \
0    2022-09-21 15:00

In [19]:
df_marked.to_csv('usersUpdated.csv')

#### OrgMember xlsx Data

In [20]:
df_orgMember = pd.read_excel('/arcgis/home/OrganizationMembers_2023-10-29.xlsx')
print(df_orgMember)

                   Username             Name                 Email  \
0           aaahern_CalPoly      Allie Ahern   aaahern@calpoly.edu   
1          aaaresta_CalPoly  Amelia Aarestad  aaaresta@calpoly.edu   
2          aabeaver_CalPoly    Audrey Beaver  aabeaver@calpoly.edu   
3             aabsy_CalPoly          AJ Absy     aabsy@calpoly.edu   
4     aabuaita_CalPoly_2023    Aydan Abuaita  aabuaita@calpoly.edu   
...                     ...              ...                   ...   
6170         ztoler_CalPoly    Zachery Toler    ztoler@calpoly.edu   
6171        ztopper_CalPoly   Zachary Topper   ztopper@calpoly.edu   
6172        zwarsen_CalPoly      Zane Warsen   zwarsen@calpoly.edu   
6173       zwillman_CalPoly     Zane Willman  zwillman@calpoly.edu   
6174       zzwahlen_CalPoly     Zach Zwahlen  zzwahlen@calpoly.edu   

     ProfileVisibility MyEsriAccess              UserType               Role  \
0              account      enabled  GISProfessionalAdvUT  account_publisher   

In [21]:
print(ps.sqldf("""SELECT *
FROM df_orgMember
WHERE itemsOwned + GroupsOwned = 0""", globals()))

                   Username            Name                 Email  \
0           aaahern_CalPoly     Allie Ahern   aaahern@calpoly.edu   
1          aabeaver_CalPoly   Audrey Beaver  aabeaver@calpoly.edu   
2             aabsy_CalPoly         AJ Absy     aabsy@calpoly.edu   
3     aabuaita_CalPoly_2023   Aydan Abuaita  aabuaita@calpoly.edu   
4            aabude_CalPoly     Arden Abude    aabude@calpoly.edu   
...                     ...             ...                   ...   
3279       zmwheele_CalPoly    Zack Wheeler  zmwheele@calpoly.edu   
3280       zpollats_CalPoly  Zach Pollatsek  zpollats@calpoly.edu   
3281          zrusk_CalPoly       Zach Rusk     zrusk@calpoly.edu   
3282       zstachow_CalPoly  Zoe Stachowski  zstachow@calpoly.edu   
3283       zwillman_CalPoly    Zane Willman  zwillman@calpoly.edu   

     ProfileVisibility MyEsriAccess              UserType               Role  \
0              account      enabled  GISProfessionalAdvUT  account_publisher   
1          